In [1]:
import numpy as np
from enum import Enum
from typing import Tuple

In [19]:
PER_STEP_REWARD = -1
ENV_DYNAMICS_PROB = 1
POLICY_PROB = 0.25
GAMMA = 0.9
EPSILON = 0.000001
IS_DETERMINISTIC = True
GRID_DIM = 4

GOAL_STATES = [(0, 0), (3, 3)]

In [20]:
coord2state = {}
state2coord = {}

st = 0
for i in range(GRID_DIM):
    for j in range(GRID_DIM):
        coord2state[(i, j)] = st
        state2coord[st] = (i, j)
        st += 1

In [21]:
print(state2coord)
print(coord2state)

GOAL_STATES_MAPPED = list(map(lambda x: coord2state[x], GOAL_STATES))
print(GOAL_STATES_MAPPED)

{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (1, 2), 7: (1, 3), 8: (2, 0), 9: (2, 1), 10: (2, 2), 11: (2, 3), 12: (3, 0), 13: (3, 1), 14: (3, 2), 15: (3, 3)}
{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6, (1, 3): 7, (2, 0): 8, (2, 1): 9, (2, 2): 10, (2, 3): 11, (3, 0): 12, (3, 1): 13, (3, 2): 14, (3, 3): 15}
[0, 15]


In [42]:
state_values = np.zeros(len(state2coord), dtype=np.float64)

for state in GOAL_STATES_MAPPED:
    state_values[state] = 0

In [43]:
class Actions(Enum):
    Up = (0, -1)
    Down = (0, 1)
    Left = (-1, 0)
    Right = (1, 0)
    
    def all_actions():
        return (Actions.Up, Actions.Down, Actions.Left, Actions.Right)

In [44]:
def apply_action(action: Actions, state: Tuple[int, int]) -> Tuple[int, int]:
    return (max(0, min(action.value[0] + state[0], GRID_DIM - 1)), max(0, min(action.value[1] + state[1], GRID_DIM - 1)))

In [45]:
max_diff = 0

while True:
    last_state_values = state_values.copy()
    max_diff = 0

    for i in range(GRID_DIM):
        for j in range(GRID_DIM):
            current_state = coord2state[(i, j)]
            if current_state in GOAL_STATES_MAPPED:
                continue

            best_value = -float('inf')
            for action in Actions.all_actions():
                next_state = coord2state[apply_action(action, state2coord[current_state])]
                value = POLICY_PROB * (PER_STEP_REWARD + GAMMA * last_state_values[next_state])
                best_value = max(best_value, value)

            state_values[current_state] = best_value
            max_diff = max(max_diff, abs(state_values[current_state] - last_state_values[current_state]))

    if max_diff < EPSILON:
        break


In [46]:
state_values = np.array(state_values, dtype=np.float64).reshape((GRID_DIM, GRID_DIM))
print(state_values)

[[ 0.         -0.25       -0.30625    -0.31890625]
 [-0.25       -0.30625    -0.31890625 -0.30625   ]
 [-0.30625    -0.31890625 -0.30625    -0.25      ]
 [-0.31890625 -0.30625    -0.25        0.        ]]
